In [1]:
#loading the model and its tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "./distilgpt2-wekeza-finetuned_v3_lora"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

C:\Users\bbollo\AppData\Local\anaconda3\envs\wekeza\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D(nf=2304, nx=768)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (c_proj): lora.Linear(
            (base_layer): Conv1D(nf=768, nx=768)
            (l

In [2]:
#text gen function
def generate_output(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [3]:
# ICL Test Example: Kenya Money Market Fund Returns as at 28th July
icl_prompt = """
Below are examples of returns for Kenyan Money Market Funds:

Example 1:
Fund: NCBA Money Market Fund
Return: 13.5%

Example 2:
Fund: Sanlam Money Market Fund
Return: 13.3%

Example 3:
Fund: CIC Money Market Fund
Return:"""

# Generate output based on prompt with examples
icl_output = generate_output(icl_prompt)
print("=== ICL Prediction ===")
print(icl_output)


=== ICL Prediction ===

Below are examples of returns for Kenyan Money Market Funds:

Example 1:
Fund: NCBA Money Market Fund
Return: 13.5%

Example 2:
Fund: Sanlam Money Market Fund
Return: 13.3%

Example 3:
Fund: CIC Money Market Fund
Return: 12.5%

Example 4:
Money Market Fund
Return: 12.5%

Example 5:
Money Market Fund
Return: 12.5%

Example 6:
Money Market Fund
Return: 12.5%

Example 7:
Money Market Fund
Return: 12.5%

Example 8:
Money Market Fund
Return: 12.5%

Example 9:
Money Market Fund
Return: 12.5%


In [4]:
#comparison
no_icl_prompt = "What is the return of CIC Money Market Fund?"
no_icl_output = generate_output(no_icl_prompt)
print("=== Non-ICL Prediction ===")
print(no_icl_output)

=== Non-ICL Prediction ===
What is the return of CIC Money Market Fund?

### Response:
CIC Money Market Fund returns annually from CIC Money Market Fund to CIC Money Market Fund. CIC Money Market Fund returns annually from CIC Money Market Fund to CIC Money Market Fund. CIC Money Market Fund returns annually from CIC Money Market Fund to CIC Money Market Fund. CIC Money Market Fund returns annually from CIC Money Market Fund to CIC Money Market Fund. CIC Money Market Fund returns annually from CIC Money Market Fund to


In [5]:
#trying new prompt
icl_prompt_natural = """
Here are some summaries of returns from popular Kenyan money market funds:

- The NCBA Money Market Fund posted an annual return of 13.5% this quarter.
- Sanlam’s Money Market Fund achieved a yield of 13.3% during the same period.
- CIC Money Market Fund recorded a return of
"""

icl_output_natural = generate_output(icl_prompt_natural)
print("=== ICL Prediction (Natural Language Style) ===")
print(icl_output_natural)

=== ICL Prediction (Natural Language Style) ===

Here are some summaries of returns from popular Kenyan money market funds:

- The NCBA Money Market Fund posted an annual return of 13.5% this quarter.
- Sanlam’s Money Market Fund achieved a yield of 13.3% during the same period.
- CIC Money Market Fund recorded a return of
12.5% in 2015.
- CIC Money Market Fund returns were held quarterly.
### Response:
CIC Money Market Fund returns are typically held quarterly, but quarterly returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual returns typically reflect quarterly returns. Annual
